# Modelo KNN

## Importando bibliotecas

In [ ]:
# Importando bibliotecas basicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# knn
from sklearn.neighbors import KNeighborsRegressor 

# Metricas de avaliaçao
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# importando biblioteca para dividir em train e test
from sklearn.model_selection import train_test_split

## Qualidade da imagem

In [ ]:
# definindo a qualidade das imagens
plt.rcParams['figure.dpi'] = 500
plt.rcParams['savefig.dpi'] = 500

## Importando os dados

In [ ]:
nomeArquivo = 'dados.xlsx'
# importa o arquivo
df_xlsx = pd.ExcelFile(nomeArquivo, engine='openpyxl')
df = pd.read_excel(df_xlsx)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Data'] = pd.to_datetime(df['Data'])
df = df.set_index(['Data'])
# remove NaN values
df = df.dropna()
# retorna df
df.head()

## KNN

In [ ]:
dfAnalise = pd.DataFrame()
# Copia os dados
dfAnalise['y'] = df['PZA']
# Cria as leituras do instante t-1
dfAnalise['x'] = dfAnalise['y'].shift(1)
dfAnalise = dfAnalise.dropna()
# retorna o df de analise
dfAnalise.head()

## Treino e teste - 60/40

In [ ]:
# dividindo o banco em treino e teste
porcentagemTreino = 0.60

porcentagemTeste = 1 - porcentagemTreino
xTreino, xTeste, yTreino, yTeste = train_test_split(dfAnalise['x'],
                                                   dfAnalise['y'],
                                                   test_size = porcentagemTeste,
                                                   random_state=21)
xTreinoReshaped = np.array(xTreino).reshape((-1, 1))
xTesteReshaped = np.array(xTeste).reshape((-1, 1))

In [ ]:
# Calibrando o modelo
# arrays para guardar os resultados
listaNumeroVizinhos = np.empty(0)
listaCoeficienteDeterminacaoTreino = np.empty(0)
listaCoeficienteDeterminicaoTeste = np.empty(0)
listaRMSETreino = np.empty(0)
listaRMSETeste = np.empty(0)
listeMAETreino = np.empty(0)
listaMAETeste = np.empty(0)
# modelo
for numeroVizinho in np.arange(2, 40, 1):
    knn = KNeighborsRegressor(n_neighbors = numeroVizinho)
    # salvando o numero de vizinhos
    listaNumeroVizinhos = np.append(listaNumeroVizinhos, numeroVizinho)
    knn.fit(xTreinoReshaped,yTreino)
    # predizendo valores
    yTreinoPredito = knn.predict(xTreinoReshaped)
    yTestePredito = knn.predict(xTesteReshaped) 
    # Avaliacao dos resultados coeficente de determinacao
    coeficienteDeterminacaoTreino = r2_score(yTreino, yTreinoPredito) 
    coeficienteDeterminacaoTeste = r2_score(yTeste, yTestePredito)
    # salvando coeficientes
    listaCoeficienteDeterminacaoTreino = np.append(listaCoeficienteDeterminacaoTreino, coeficienteDeterminacaoTreino)
    listaCoeficienteDeterminicaoTeste = np.append(listaCoeficienteDeterminicaoTeste, coeficienteDeterminacaoTeste)
    # RMSE
    rmseTreino = mean_squared_error(yTreino,yTreinoPredito,squared=False) 
    rmseTeste = mean_squared_error(yTeste,yTestePredito,squared=False) 
    listaRMSETreino = np.append(listaRMSETreino, rmseTreino)
    listaRMSETeste = np.append(listaRMSETeste, rmseTeste)
    #MAE
    maeTreino = mean_absolute_error(yTreino,yTreinoPredito) 
    maeTeste = mean_absolute_error(yTeste,yTestePredito)
    listeMAETreino = np.append(listeMAETreino, maeTreino)
    listaMAETeste = np.append(listaMAETeste, maeTeste)
# grafico de calibracao
fig, ax1 = plt.subplots(figsize=(5,5))
plt.scatter(listaNumeroVizinhos, listaCoeficienteDeterminacaoTreino, label='Treino')
plt.scatter(listaNumeroVizinhos, listaCoeficienteDeterminicaoTeste, label='Teste')
plt.legend()
plt.xlabel('Número de vizinhos')
plt.ylabel('Coeficiente de determinação')
plt.title('Avaliação do número de vizinhos - Distância Euclidiana')
plt.show();

In [ ]:
# diferenca entre os coeficientes
listaDiferencaCoeficientes = listaCoeficienteDeterminacaoTreino - listaCoeficienteDeterminicaoTeste
# grafico
plt.scatter(listaNumeroVizinhos, listaDiferencaCoeficientes)
plt.xlabel('Número de vizinhos')
plt.ylabel('Diferença entres os coeficientes de determinação de treino e teste')
plt.title('Avaliação do número de vizinhos - Distância Euclidiana')
plt.show();

In [ ]:
# Modelo

# iniciando o modelo
knn = KNeighborsRegressor(n_neighbors = 10)
# treinando o modelo
knn.fit(xTreinoReshaped,yTreino)

# Avaliando o modelo
# predizendo valores
yTreinoPredito = knn.predict(xTreinoReshaped)
yTestePredito = knn.predict(xTesteReshaped) 
# Avaliacao dos resultados coeficente de determinacao
coeficienteDeterminacaoTreino = r2_score(yTreino, yTreinoPredito) 
print(f"O coeficiente de determinação do banco de treino é {coeficienteDeterminacaoTreino}")
coeficienteDeterminacaoTeste = r2_score(yTeste, yTestePredito)
print(f"O coeficiente de determinação do banco de teste é {coeficienteDeterminacaoTeste}")
print('-------------------------------------')
# Avaliacao dos resultados RMSE
rmseTreino = mean_squared_error(yTreino,yTreinoPredito,squared=False) 
print(f"O RMSE do banco de treino é {rmseTreino}")
rmseTeste = mean_squared_error(yTeste,yTestePredito,squared=False) 
print(f"O RMSE do banco de teste é {rmseTeste}")
print('-------------------------------------')
# Avaliacao dos resultados MAE
maeTreino = mean_absolute_error(yTreino,yTreinoPredito) 
print(f"O MAE do banco de treino é {maeTreino}")
maeTeste = mean_absolute_error(yTeste,yTestePredito)
print(f"O MAE do banco de teste é {maeTeste}")

In [ ]:
# grafico de autocorrelacao dos instantes t e t-1
fig, ax1 = plt.subplots(figsize = (5, 5))
sns.scatterplot(x='x', y='y', data=dfAnalise, ax=ax1)
ax1.scatter(xTreino, yTreinoPredito, color='red')
ax1.scatter(xTeste, yTestePredito, color='green')
plt.ylabel('Instante t')
plt.xlabel('Instante t-1');